<a href="https://colab.research.google.com/github/flydevilz/automate_zoom_attendance/blob/main/Check_Presensi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Use:
1. Download CSV di ZOOM, dan **jangan centang Export Data Meeting**. Setelah itu masukkan csv di GDrive masing2.
2. Set share -> general access -> anyone with the link
3. Copy link dan paste di bagian Import Data -> `link attendance`. Ubah ke section masing-masing.
4. Runtime -> Run All
5. Yg ditutup gausah dibuka yaa, yg dibutuhkan cuma bagian "Tambah Presensi", "Tidak Hadir" dan "Ga ke detect" yaa((:

Note: untuk bagian  `link attendance` kalian pakai file terpisah dari csv yg di programnya ka Dew (soalnya yg di spreadsheet kan downloadnya file csv zoom nya pakai export data meeting).
Program disini baru bisa jalan kalau csv nya gak di export data meetingnya (:
Jadi kalau mau pake ini harus download lg file csv zoom nya yg gk pakai export data meeting (yg disuru centang kemaren, jgn di centang). Tunggu update patch berikutnya ygy baru bisa hehe(:


# Import Data

In [8]:
import pandas as pd
import numpy as np
import re

In [9]:
insert_link= 'https://drive.google.com/file/d/13hByEk-bsoeYniawokVpvI1LWCYq3hEY/view?usp=drive_link'
pattern_link = r"https://drive\.google\.com/file/d/([\w-]+)/view\?usp=drive_link"
replacement = r"https://drive.google.com/uc?id=\1"

In [10]:
# ubah link attendance jd link spreadsheet masing2
link_airtable= 'https://drive.google.com/uc?id=1gIKKnVxugWQd_WxuOyGNSQRuPli_AOfw'
link_attendance = re.sub(pattern_link, replacement, insert_link)
link_level3= 'https://drive.google.com/uc?id=1JrZp_Ve6mekR-dPqR8xQBAlrgZtBjXqw'

Kalau error, kemungkinan bentuk linknya masih kaya gini:
https://drive.google.com/file/d/1gIKKnVxugWQd_WxuOyGNSQRuPli_AOfw/view?usp=drive_link
dihapus aja `/file/d/` dalam link diganti jadi `/uc?id=` , terus `/view?usp=drive_link` dihapus aja.

In [11]:
airtable = pd.read_csv(link_airtable)
attendance = pd.read_csv(link_attendance)
level3 = pd.read_csv(link_level3, skiprows=2)

# Clean Airtable

In [12]:
# ubah nama section
airtable_semarang = airtable[(airtable['Section'] == 'Semarang')]

In [13]:
# filter kolom yg dibutuhkan
airtable_name_email = airtable_semarang[['Student ID', 'Student Name', 'Email', 'Email RevoU']]

In [14]:
# rename column nama supaya jelas
airtable_name_email = airtable_name_email.rename(columns= {'Student ID':'id','Student Name': 'name', 'Email RevoU':'email_revou'})

In [15]:
# clean double space di nama
airtable_name_email['name'] = airtable_name_email['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
airtable_name_email['name'] = airtable_name_email['name'].str.upper()

<ipython-input-15-accc58cb7812>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  airtable_name_email['name'] = airtable_name_email['name'].str.strip().str.replace(r'\s+', ' ')


In [16]:
# ubah data type Student ID supaya gampang di merge
airtable_name_email['id'] = airtable_name_email['id'].astype('int64')

# Clean Attendance Zoom

In [17]:
# rename column nama supaya jelas
attendance_name_email = attendance.rename(columns= {'Name (Original Name)':'name','User Email': 'email', 'Duration (Minutes)':'duration'})

In [18]:
# filter kolom yg dibutuhkan
attendance_name_email = attendance_name_email[['name', 'email', 'duration']]

In [19]:
# clean double space di nama
attendance_name_email['name'] = attendance_name_email['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
attendance_name_email['name'] = attendance_name_email['name'].str.upper()

<ipython-input-19-6feaf5f74b62>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  attendance_name_email['name'] = attendance_name_email['name'].str.strip().str.replace(r'\s+', ' ')


# Clean Level 3

In [20]:
level3.dropna(subset=['Student ID'], inplace=True)

In [21]:
# ubah data type supaya gampang di merge
level3['Student ID'] = level3['Student ID'].astype('int64')
level3['Team'] = level3['Team'].astype('int64')

In [22]:
# rename column nama supaya jelas
level3 = level3.rename(columns= {'Student ID':'id','Student Name': 'name', 'Email':'email'})

In [23]:
# clean double space di nama
level3['name'] = level3['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
level3['name'] = level3['name'].str.upper()

<ipython-input-23-8e8b720d35b6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  level3['name'] = level3['name'].str.strip().str.replace(r'\s+', ' ')


In [24]:
# filter kolom yg dibutuhkan
absensi = level3.iloc[:, [0, 1, 2, 3, -1]]

In [25]:
absensi = absensi[['Team','name','email']]

## Check email kalau udah sama dengan airtable

In [26]:
dum_lv3= level3[['id','email']]

In [27]:
dum_email_air = airtable_name_email[['id','name', 'email_revou']]

In [28]:
check = pd.merge(dum_email_air, dum_lv3, on='id', how='left', suffixes = ('_airtable', '_level3'))

In [29]:
check.head()

,id,name,email_revou,email
0,7458741,DEVIANA PUSPITA,devpus89@gmail.com,devpus89@gmail.com
1,7460303,VIRGO JUNIOR SITEPU,virgojuniorsitepu@gmail.com,virgojuniorsitepu@gmail.com
2,7470736,FAURIKA,safaurika@gmail.com,safaurika@gmail.com
3,7471297,ARIANTO TAMBUNAN,tambunanarianto4@gmail.com,tambunanarianto4@gmail.com
4,7478722,FIRYAL AZKIATUL IFTINAN,firyalazkiatul04@gmail.com,firyalazkiatul04@gmail.com


In [30]:
check['Notes'] = check['email_revou'] == check['email']

In [31]:
mismatched_email = check[check['Notes'] == False]

In [32]:
mismatched_email

,id,name,email_revou,email,Notes


Kalau sama berarti aman :) Lanjut step berikut

# Next Step

In [33]:
attendance_name_email.shape

(662, 3)

In [34]:
attendance_name_email['email'].fillna('-', inplace=True)

In [35]:
total_duration = attendance_name_email.groupby(['name', 'email'], as_index=False)['duration'].sum()

In [36]:
att_check = pd.merge(absensi, total_duration, how='outer', on = 'email', suffixes= ('_level3', '_zoom'))

In [37]:
att_check.shape

(392, 5)

In [38]:
condition_1 = (att_check['name_level3'].isnull()) & (att_check['name_zoom'].notnull())
hadir_zoom_ga_kedetect = att_check[condition_1]
hadir_zoom_ga_kedetect = hadir_zoom_ga_kedetect[['name_zoom', 'duration']]

In [39]:
condition_2 = (att_check['name_level3'].notnull()) & (att_check['name_zoom'].isnull())
yg_mau_dimerge = att_check[condition_2]
yg_mau_dimerge = yg_mau_dimerge[['Team','name_level3','email']]

In [40]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Yang join ZOOM

In [41]:
right = pd.merge(yg_mau_dimerge, hadir_zoom_ga_kedetect, left_on = 'name_level3', right_on= 'name_zoom', how='right')

In [42]:
condition_3 = (right['name_level3'].notnull()) & (right['email'].notnull()) & (right['name_zoom'].notnull())
fix = right[condition_3]

# TAMBAH PRESENSI (YG GA SESUAI EMAIL TAPI SAMA NAMA) AKA MATCH BY NAME YG DIRUASINYA >=60

In [43]:
# @title
sorted_fix = fix.sort_values(by='duration')
hadir_diatas_60 = sorted_fix[sorted_fix['duration'] >= 60]

In [44]:
hadir_diatas_60.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
44,1.0,MUHAMMAD FATAN BASAR,fatanbasar@gmail.com,MUHAMMAD FATAN BASAR,154.0
65,1.0,SELYA FAJRINA,fajrinaaaselyaaa@gmail.com,SELYA FAJRINA,140.0
53,2.0,PERWITA PUSPASARI,perwita592@gmail.com,PERWITA PUSPASARI,140.0
6,2.0,ANANDA RAHMI FARIKHA,aanandarahmi@gmail.com,ANANDA RAHMI FARIKHA,136.0
50,2.0,NATASHA FARAS NABILA,tashafn343@gmail.com,NATASHA FARAS NABILA,152.0
11,3.0,ARVIN CHIENDRA WIJAYA,arvinwijayaxx@gmail.com,ARVIN CHIENDRA WIJAYA,133.0
4,3.0,AHMAD AKBAR,ahmadakbar21206@gmail.com,AHMAD AKBAR,152.0
76,4.0,WILDAN FIRDAUS,wildanf14.school@gmail.com,WILDAN FIRDAUS,134.0
80,4.0,ALIFA BAIDURI HAYATUNNUFUS,alifabaiduri7@gmail.com,ALIFA BAIDURI HAYATUNNUFUS,95.0
29,4.0,FEYZA PUTRI AULIA PRATAMI,feyzapap@gmail.com,FEYZA PUTRI AULIA PRATAMI,127.0


# TIDAK HADIR (GADA EMAIL TAPI SAMA NAMA) <60 DURASI

In [45]:
# @title
hadir_dibawah_60 = sorted_fix[sorted_fix['duration'] < 60]

In [46]:
hadir_dibawah_60.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
41,4.0,MUHAMMAD FADLI ISYARQI,siyarqifadli2@gmail.com,MUHAMMAD FADLI ISYARQI,19.0
2,9.0,ADIS FUTRI MARESYA,aadismaresyaa@gmail.com,ADIS FUTRI MARESYA,39.0
52,14.0,NURUL ROFIDHATUL ILMI,nurul.rofidhatul112@gmail.com,NURUL ROFIDHATUL ILMI,50.0


# Yg harus di RO (Merge program di Spreadsheet dan kode di atas)

In [47]:
# @title
a = att_check[~(condition_1 | condition_2)]
b = pd.concat([a, fix], ignore_index=True)

In [48]:
print(f'jumlah student yg berhasil ke detect: {b.shape[0]} akun. \n Note: ini hanya student. Tidak termasuk SM dan Mentor.')

jumlah student yg berhasil ke detect: 233 akun. 
 Note: ini hanya student. Tidak termasuk SM dan Mentor.


In [49]:
# @title
# yg di bawah 60 menit:
sorted_fix = b.sort_values(by='duration')
filtered_fix = sorted_fix[sorted_fix['duration'] < 60]

In [50]:
print(f'jumlah student yg harus di RO buat makesure: {filtered_fix.shape[0]} student.')

jumlah student yg harus di RO buat makesure: 33 student.


In [67]:
filtered_fix.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
7,2.0,ADIT PRAYOGA,aditprayogaa02@gmail.com,ADIT PRAYOGA,16.0
23,4.0,ALLIF RIZKI ABDILLAH,allifrizki02@gmail.com,ALLIF RIZKI,1.0
209,4.0,MUHAMMAD FADLI ISYARQI,siyarqifadli2@gmail.com,MUHAMMAD FADLI ISYARQI,19.0
30,5.0,RANGGA BHAGASKARA,ranggabhagaskaraa@gmail.com,RANGGA BHAGASKARA,8.0
34,6.0,RIZKY RAMADHAN,kiki.adonk@gmail.com,RIZKY RAMADHAN,30.0
39,6.0,LIA KUSUMA WARDANI,liakusumaw96@gmail.com,LIA KUSUMA,15.0
47,7.0,SOFFADINA AUFA RAHMA,soffadina89@gmail.com,SOFFADINA AUFA RAHMA (SOFFA DINA),17.0
192,9.0,ADIS FUTRI MARESYA,aadismaresyaa@gmail.com,ADIS FUTRI MARESYA,39.0
63,9.0,SYARADELLA NURMUTHASYA,syaradella@gmail.com,SYARADELLA NURMUTHASYA,9.0
59,9.0,RAGIL SITI ISTIQOMAH,ragilsitiistiqomah512@gmail.com,RAGIL SITI ISTIQOMAH,22.0


In [52]:
# @title
def check_name_di_merge(name):
  contains_name = (filtered_fix['name_zoom'].str.contains(name, na=False)) | (filtered_fix['name_level3'].str.contains(name, na=False))
  return filtered_fix[contains_name]

In [68]:
# buat make sure di check aja namanya ada ga
result = check_name_di_merge('ADINDA')
result

,Team,name_level3,email,name_zoom,duration
101,13.0,ADINDA SALSABILA,adindaaasals28@gmail.com,210017 - ADINDA SALSABILA,57.0


# Yang Ga Ke Detect

In [54]:
# @title
outer = pd.merge(yg_mau_dimerge, hadir_zoom_ga_kedetect, left_on = 'name_level3', right_on= 'name_zoom', how='outer')

In [55]:
# @title
condition_4 = (outer['name_level3'].notnull()) & (outer['email'].notnull())
f = outer[~(condition_4)]

In [56]:
f.shape

(43, 5)

In [57]:
f

,Team,name_level3,email,name_zoom,duration
116,NaN,NaN,NaN,2_KHARAZ TSANIYA,5.0
117,NaN,NaN,NaN,ADELIA TRI,175.0
118,NaN,NaN,NaN,ADIT (TE_2003025047_REZAPRAYUDHA),94.0
119,NaN,NaN,NaN,AKMAL,169.0
120,NaN,NaN,NaN,ANISA DEWI,109.0
121,NaN,NaN,NaN,ARGA ADYATAMA,1.0
122,NaN,NaN,NaN,ARGA ADYATAMA - MENTOR (ARGA ADYATAMA),153.0
123,NaN,NaN,NaN,CLAUDIO,45.0
124,NaN,NaN,NaN,CLAUDIO (DIY),1.0
125,NaN,NaN,NaN,DAVID JUNIOR,8.0


In [58]:
pattern = r'^\d+_([A-Z\s]+)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern, r'\1', regex=True)
pattern2 = r'^\d+\s+([A-Z\s]+)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern2, r'\1', regex=True)

<ipython-input-58-5ffb9168a6e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern, r'\1', regex=True)
<ipython-input-58-5ffb9168a6e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern2, r'\1', regex=True)


In [59]:
total_duration1 = f.groupby(['name_zoom'], as_index=False)['duration'].sum()

In [60]:
f.dropna(subset=['name_zoom', 'duration'], inplace=True)

<ipython-input-60-a0f4cd478d51>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.dropna(subset=['name_zoom', 'duration'], inplace=True)


In [61]:
pattern3 = r'\((.*?)\)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern3, '', regex=True).str.strip()

<ipython-input-61-40ad52d32664>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern3, '', regex=True).str.strip()


In [62]:
total_duration1 = f.groupby(['name_zoom'], as_index=False)['duration'].sum()

In [63]:
total_duration1

,name_zoom,duration
0,ADELIA TRI,175.0
1,ADIT,94.0
2,AGIT,180.0
3,AKMAL,169.0
4,ANISA DEWI,109.0
5,ARGA ADYATAMA,1.0
6,ARGA ADYATAMA - MENTOR,153.0
7,CLAUDIO,46.0
8,DAVID JUNIOR,8.0
9,DIY,41.0


In [64]:
f

,Team,name_level3,email,name_zoom,duration
116,NaN,NaN,NaN,KHARAZ TSANIYA,5.0
117,NaN,NaN,NaN,ADELIA TRI,175.0
118,NaN,NaN,NaN,ADIT,94.0
119,NaN,NaN,NaN,AKMAL,169.0
120,NaN,NaN,NaN,ANISA DEWI,109.0
121,NaN,NaN,NaN,ARGA ADYATAMA,1.0
122,NaN,NaN,NaN,ARGA ADYATAMA - MENTOR,153.0
123,NaN,NaN,NaN,CLAUDIO,45.0
124,NaN,NaN,NaN,CLAUDIO,1.0
125,NaN,NaN,NaN,DAVID JUNIOR,8.0


In [65]:
# run this to check if there's some name you want to check
def check_name(name):
  contains_name = (f['name_zoom'].str.contains(name, na=False)) | (filtered_fix['name_zoom'].str.contains(name, na=False)) | (hadir_diatas_60['name_zoom'].str.contains(name, na=False))
  return filtered_fix[contains_name]

In [66]:
# and this, just change the name
result = check_name('NAMIRAH')
result

<ipython-input-65-01633e7e33ff>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return filtered_fix[contains_name]


,Team,name_level3,email,name_zoom,duration
158,20.0,DINI NURHINDA MAHARANI,ranidini766@gmail.com,DINI NM,23.0
